In [1]:
import cv2 
import numpy as np
import mtcnn
from scipy.spatial.distance import cosine
from tensorflow.keras.models import load_model
import pickle
import sys
sys.path.append('..')

from facenet.architecture import *
from facenet.preprocessing import normalize,l2_normalizer

import os
while not os.getcwd().endswith('_det'.lower()):
    os.chdir('..')
print(os.getcwd())


confidence_t=0.99
recognition_t=0.5
required_size = (160,160)

def get_face(img, box):
    x1, y1, width, height = box
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = img[y1:y2, x1:x2]
    return face, (x1, y1), (x2, y2)

def get_encode(face_encoder, face, size):
    face = normalize(face)
    face = cv2.resize(face, size)
    encode = face_encoder.predict(np.expand_dims(face, axis=0))[0]
    return encode

def load_pickle(path):
    with open(path, 'rb') as f:
        encoding_dict = pickle.load(f)
    return encoding_dict

def detect(img ,detector,encoder,encoding_dict):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(img_rgb)
    for res in results:
        if res['confidence'] < confidence_t:
            continue
        face, pt_1, pt_2 = get_face(img_rgb, res['box'])
        encode = get_encode(encoder, face, required_size)
        encode = l2_normalizer.transform(encode.reshape(1, -1))[0]
        name = 'unknown'

        distance = float("inf")
        for db_name, db_encode in encoding_dict.items():
            dist = cosine(db_encode, encode)
            if dist < recognition_t and dist < distance:
                name = db_name
                distance = dist

        if name == 'unknown':
            cv2.rectangle(img, pt_1, pt_2, (0, 0, 255), 2)
            # cv2.putText(img, name, pt_1, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)
            continue
        else:
            cv2.rectangle(img, pt_1, pt_2, (0, 255, 0), 2)
            cv2.putText(img, name + f'__{distance:.2f}', (pt_1[0], pt_1[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 200, 200), 2)
    return img 


d:\Uni\Masters\CS604 DL for Visual Recognition\Project\supermarket_det


In [2]:
required_shape = (160,160)
face_encoder = InceptionResNetV2()
path_m = "facenet/facenet_keras_weights.h5"
face_encoder.load_weights(path_m)
encodings_path = 'facenet/encodings/encodings.pkl'
face_detector = mtcnn.MTCNN()
encoding_dict = load_pickle(encodings_path)


In [6]:

required_shape = (160,160)
face_encoder = InceptionResNetV2()
path_m = "facenet/facenet_keras_weights.h5"
face_encoder.load_weights(path_m)
encodings_path = 'facenet/encodings/encodings.pkl'
face_detector = mtcnn.MTCNN()
encoding_dict = load_pickle(encodings_path)

cap = cv2.VideoCapture('data/Prof.mov')
cap.set(cv2.CAP_PROP_BUFFERSIZE,2)
f = 0

while cap.isOpened():
    ret,frame = cap.read()

    if not ret:
        print("CAM NOT OPEND") 
        break
    
    frame= detect(frame , face_detector , face_encoder , encoding_dict)

    # if f%10 == 0:
    cv2.imshow('camera', frame)
        
    FPS = 1/30
    FPS_MS = int(FPS * 1000)

    if cv2.waitKey(FPS_MS) & 0xFF == ord('q'):
        break
    f+=1

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()



